In [2]:
import pandas as pd
import numpy as np

In [14]:
# 유저테이블 불러오기
click = pd.read_csv("/Users/sparta/Documents/GitHub/final_project/data/유저테이블.csv")

# media portfolio 불러오기
media_portfolio = pd.read_csv("/Users/sparta/Documents/GitHub/final_project/data/media_portfolio.csv")

# pool 불러오기
pool = pd.read_csv("/Users/sparta/Documents/GitHub/final_project/data/ads_pool.csv")

In [55]:
media_performance = pd.read_csv("/Users/sparta/Documents/GitHub/final_project/data/media_performance_classification.csv")

In [5]:
def analyze_ads_performance(ads_idx, click_data, media_portfolio=None):
    """
    특정 광고의 매체별 성과를 분석하는 함수
    """
    
    # 1. 해당 광고의 데이터가 있는지 확인
    ads_data = click_data[click_data['ads_idx'] == ads_idx]
    if len(ads_data) == 0:
        print(f"광고 {ads_idx}에 대한 데이터가 없습니다.")
        return pd.DataFrame()
    
    # 2. 기본 성과 데이터 추출
    ads_performance = ads_data.groupby(['ads_idx', 'mda_idx']).agg({
        'click_key': 'count',
        'conversion': 'sum',
        'contract_price': 'first',
        'media_price': 'first',
        'domain': 'first',
        'ads_category': 'first'
    }).reset_index()
    
    # 컬럼명 변경
    ads_performance.columns = ['ads_idx', 'mda_idx', 'total_clicks', 'total_conversions', 
                              'contract_price', 'media_price', 'domain', 'ads_category']
    
    # 전환율 및 수익 계산
    ads_performance['cvr'] = (
        ads_performance['total_conversions'] / ads_performance['total_clicks']
    ).round(4)
    
    ads_performance['profit_per_conversion'] = (
        ads_performance['contract_price'] - ads_performance['media_price']
    )
    ads_performance['total_profit'] = (
        ads_performance['total_conversions'] * ads_performance['profit_per_conversion']
    )
    
    # 3. 날짜 범위 및 활동일 계산
    click_data_copy = click_data.copy()
    if not pd.api.types.is_datetime64_any_dtype(click_data_copy['click_date']):
        click_data_copy['click_date'] = pd.to_datetime(click_data_copy['click_date'])
    
    ads_activity = (
        click_data_copy.loc[click_data_copy['ads_idx'] == ads_idx]
                      .groupby('mda_idx')['click_date']
                      .agg(first_click='min', last_click='max')
                      .reset_index()
    )
    
    ads_activity['days_active_calc'] = (
        (ads_activity['last_click'] - ads_activity['first_click']).dt.days + 1
    )
    
    # 4. 데이터 병합
    merged = ads_performance.merge(
        ads_activity[['mda_idx', 'first_click', 'last_click', 'days_active_calc']],
        on='mda_idx', how='left'
    )
    
    # 5. 일평균 지표 계산
    merged['daily_clicks'] = merged['total_clicks'] / merged['days_active_calc']
    merged['daily_conversions'] = merged['total_conversions'] / merged['days_active_calc']
    merged['daily_profit'] = merged['total_profit'] / merged['days_active_calc']
    
    # 6. 배분 그룹 분류 (데이터가 충분한 경우에만)
    if len(merged) > 1:  # 최소 2개 이상의 매체가 있어야 중앙값 계산이 의미있음
        profit_median = merged['daily_profit'].median()
        conv_median = merged['daily_conversions'].median()
        
        merged['배분그룹'] = np.where(
            (merged['daily_profit'] >= profit_median) & (merged['daily_conversions'] >= conv_median),
            '잘 배분',
            '잘못 배분'
        )
        # 결과 정렬
        result = merged.sort_values(['배분그룹', 'daily_profit'], ascending=[True, False]).reset_index(drop=True)
    else:
        merged['배분그룹'] = '분류불가'
        result = merged.reset_index(drop=True)
    
    return result

In [39]:
ads_446909_analysis = analyze_ads_performance(28947, click)

In [40]:
ads_446909_analysis

,ads_idx,mda_idx,total_clicks,total_conversions,contract_price,media_price,domain,ads_category,cvr,profit_per_conversion,total_profit,first_click,last_click,days_active_calc,daily_clicks,daily_conversions,daily_profit,배분그룹
0,28947,294,165,53,230,180,채용,3,0.3212,50,2650,2025-07-27 19:19:34,2025-08-25 08:53:19,29,5.689655172414,1.827586206897,91.379310344828,잘 배분
1,28947,246,1,1,230,180,채용,3,1.0000,50,50,2025-08-07 15:08:24,2025-08-07 15:08:24,1,1.000000000000,1.000000000000,50.000000000000,잘 배분
2,28947,270,58,12,230,126,채용,3,0.2069,104,1248,2025-07-28 00:33:28,2025-08-24 11:03:29,28,2.071428571429,0.428571428571,44.571428571429,잘 배분
3,28947,337,7,6,230,180,채용,3,0.8571,50,300,2025-08-06 10:04:03,2025-08-21 11:11:04,16,0.437500000000,0.375000000000,18.750000000000,잘 배분
4,28947,54,25,6,230,180,채용,3,0.2400,50,300,2025-07-29 20:03:22,2025-08-22 15:59:09,24,1.041666666667,0.250000000000,12.500000000000,잘못 배분
5,28947,761,37,7,230,180,채용,3,0.1892,50,350,2025-07-26 23:51:34,2025-08-24 20:47:05,29,1.275862068966,0.241379310345,12.068965517241,잘못 배분
6,28947,30,7,5,230,180,채용,3,0.7143,50,250,2025-07-27 20:58:53,2025-08-17 02:31:44,21,0.333333333333,0.238095238095,11.904761904762,잘못 배분
7,28947,1037,2,0,230,180,채용,3,0.0000,50,0,2025-08-21 17:16:53,2025-08-21 17:18:16,1,2.000000000000,0.000000000000,0.000000000000,잘못 배분


In [10]:
ads_time = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/ads_pool.csv')
print(f"데이터 수 : {ads_time.shape}")
ads_time.head()

데이터 수 : (4714, 44)


,Unnamed: 0.1,Unnamed: 0,ads_idx,media_count,user_count,total_clicks,total_conversions,ads_category,domain,ads_os_type,...,conv_score,clicks_score,stability_score,cvr_score,total_score,ads_size,cluster,mda_idx_arr,M,A
0,0,0,6508,1,1,2,0,10,금융,7,...,0,0,0,0,1,SMALL,0,NaN,0.0,1.0
1,1,1,6985,2,30,46,0,7,금융,7,...,0,0,3,0,5,MEDIUM,0,NaN,0.0,0.0
2,2,2,8327,1,6,11,0,10,미디어/컨텐츠,7,...,0,0,3,0,4,SMALL,0,NaN,0.0,1.0
3,3,3,9264,1,1,1,0,10,금융,7,...,0,0,0,0,1,SMALL,0,NaN,0.0,1.0
4,4,4,9716,2,4,6,1,8,생활,2,...,0,1,1,0,4,SMALL,0,NaN,0.0,1.0


In [17]:
pool[pool['ads_idx'] == 157236]

,Unnamed: 0.1,Unnamed: 0,ads_idx,media_count,user_count,total_clicks,total_conversions,ads_category,domain,ads_os_type,...,conv_score,clicks_score,stability_score,cvr_score,total_score,ads_size,cluster,mda_idx_arr,M,A
449,461,481,157236,7,119,167,72,4,식당/카페,7,...,1,1,3,0,8,LARGE,0,"562,563",1.0,1.0


In [62]:
pd.set_option('display.max_rows', None)        # 모든 행 표시
pd.set_option('display.max_columns', None)     # 모든 열 표시
pd.set_option('display.width', 0)              # 가로폭 자동(줄바꿈 안 함)
pd.set_option('display.max_colwidth', None)    # 긴 텍스트 자르지 않음
pd.set_option('display.precision', 12)  

In [42]:
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.width')
pd.reset_option('display.max_colwidth')
pd.reset_option('display.precision')
pd.options.display.float_format = None
np.set_printoptions(threshold=1000, linewidth=75)


In [43]:
pool[pool['ads_size'] == 'LARGE']

,Unnamed: 0.1,Unnamed: 0,ads_idx,media_count,user_count,total_clicks,total_conversions,ads_category,domain,ads_os_type,...,conv_score,clicks_score,stability_score,cvr_score,total_score,ads_size,cluster,mda_idx_arr,M,A
5,5,5,9935,10,524,889,24,8,금융,7,...,0,2,4,0,9,LARGE,0,NaN,0.0,1.0
26,29,29,13209,14,758,2162,169,8,금융,2,...,1,3,4,0,12,LARGE,0,22,1.0,1.0
27,30,30,13927,2,2069,2808,1032,1,생활,1,...,3,3,3,0,11,LARGE,0,NaN,0.0,1.0
31,34,34,14074,15,186,558,33,8,금융,2,...,1,2,4,0,11,LARGE,0,NaN,0.0,1.0
34,37,37,14405,9,317,505,40,10,미디어/컨텐츠,7,...,1,2,4,0,10,LARGE,0,NaN,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4701,4758,9504,446891,2,27,30,25,4,뷰티,7,...,2,2,0,2,8,LARGE,0,NaN,0.0,0.0
4704,4761,9507,446894,2,20,22,17,4,커머스,7,...,2,2,0,2,8,LARGE,0,NaN,0.0,0.0
4706,4763,9509,446896,2,39,48,39,4,커머스,7,...,2,2,0,2,8,LARGE,0,NaN,0.0,0.0
4709,4766,9512,446899,2,52,63,49,4,커머스,7,...,3,2,0,2,9,LARGE,0,NaN,0.0,0.0


In [41]:
def analyze_ads_performance(ads_idx, click_data, media_portfolio=None, mda_idx=None):
    """
    특정 광고/매체 조합의 매체별 성과를 분석
    - ads_idx: 광고 ID (None이면 무시)
    - mda_idx: 매체사 ID (None이면 무시)
    """
    df = click_data.copy()

    # 1) 필터링: 광고, 매체 중 지정된 것만 적용
    if ads_idx is not None and not (isinstance(ads_idx, float) and pd.isna(ads_idx)):
        df = df[df['ads_idx'] == int(ads_idx)]
    if mda_idx is not None and not (isinstance(mda_idx, float) and pd.isna(mda_idx)):
        df = df[df['mda_idx'] == int(mda_idx)]

    if len(df) == 0:
        target = f"ads_idx={ads_idx}" if ads_idx is not None else ""
        target += f" mda_idx={mda_idx}" if mda_idx is not None else ""
        print(f"해당 조건({target.strip()})에 대한 데이터가 없습니다.")
        return pd.DataFrame()

    # 2) 기본 성과 집계 (항상 ads_idx × mda_idx 단위)
    agg = df.groupby(['ads_idx', 'mda_idx']).agg(
        total_clicks   = ('click_key', 'count'),
        total_conversions = ('conversion', 'sum'),
        contract_price = ('contract_price', 'first'),
        media_price    = ('media_price', 'first'),
        domain         = ('domain', 'first'),
        ads_category   = ('ads_category', 'first'),
    ).reset_index()

    # 전환율/수익
    agg['cvr'] = (agg['total_conversions'] / agg['total_clicks']).replace([np.inf, -np.inf], np.nan).fillna(0).round(4)
    agg['profit_per_conversion'] = (agg['contract_price'] - agg['media_price'])
    agg['total_profit'] = agg['total_conversions'] * agg['profit_per_conversion']

    # 3) 활동 기간(first/last click) 및 일평균 지표
    if not pd.api.types.is_datetime64_any_dtype(df['click_date']):
        df['click_date'] = pd.to_datetime(df['click_date'], errors='coerce')

    act = (df.groupby(['ads_idx','mda_idx'])['click_date']
             .agg(first_click='min', last_click='max')
             .reset_index())
    act['days_active_calc'] = (act['last_click'] - act['first_click']).dt.days + 1
    act['days_active_calc'] = act['days_active_calc'].clip(lower=1)  # 0일 방지

    merged = agg.merge(act, on=['ads_idx','mda_idx'], how='left')

    merged['daily_clicks']      = merged['total_clicks']      / merged['days_active_calc']
    merged['daily_conversions'] = merged['total_conversions'] / merged['days_active_calc']
    merged['daily_profit']      = merged['total_profit']      / merged['days_active_calc']

    # 4) (옵션) media_portfolio 붙이기
    if media_portfolio is not None and 'mda_idx' in media_portfolio.columns:
        merged = merged.merge(media_portfolio, on='mda_idx', how='left')

    # 5) 배분그룹 (행이 2개 이상일 때만)
    if len(merged) > 1:
        profit_median = merged['daily_profit'].median()
        conv_median   = merged['daily_conversions'].median()
        merged['배분그룹'] = np.where(
            (merged['daily_profit'] >= profit_median) & (merged['daily_conversions'] >= conv_median),
            '잘 배분', '잘못 배분'
        )
        merged = merged.sort_values(['배분그룹','daily_profit'], ascending=[True, False]).reset_index(drop=True)
    else:
        merged['배분그룹'] = '분류불가'
        merged = merged.reset_index(drop=True)

    return merged

# 편의용: 매체사 ID만으로 분석 (여러 광고가 그 매체에 집행된 경우 광고별로 한 줄씩 나옵니다)
def analyze_media_performance(mda_idx, click_data, media_portfolio=None, ads_idx=None):
    """
    mda_idx 기준 분석 (ads_idx를 추가로 주면 특정 광고만 필터)
    """
    return analyze_ads_performance(
        ads_idx=ads_idx,
        click_data=click_data,
        media_portfolio=media_portfolio,
        mda_idx=mda_idx
    )


In [59]:
# 1) 기존처럼 광고 기준
ads_28947 = analyze_ads_performance(57056, click)

In [60]:
ads_28947

,ads_idx,mda_idx,total_clicks,total_conversions,contract_price,media_price,domain,ads_category,cvr,profit_per_conversion,total_profit,first_click,last_click,days_active_calc,daily_clicks,daily_conversions,daily_profit,배분그룹
0,57056,14,1574,117,230,180,금융,1,0.0743,50,5850,2025-07-26 00:23:06,2025-08-25 09:15:47,31,50.774194,3.774194,188.709677,잘 배분
1,57056,458,41,10,230,180,금융,1,0.2439,50,500,2025-07-26 00:44:35,2025-08-25 02:40:36,31,1.322581,0.322581,16.129032,잘 배분
2,57056,54,17,3,230,180,금융,1,0.1765,50,150,2025-07-26 14:58:30,2025-08-24 10:58:54,29,0.586207,0.103448,5.172414,잘못 배분
3,57056,761,38,2,230,180,금융,1,0.0526,50,100,2025-07-28 08:32:14,2025-08-24 11:09:26,28,1.357143,0.071429,3.571429,잘못 배분


In [ ]:
# 3) 특정 광고 + 특정 매체만 보고 싶을 때(정확히 한 줄 나옴)
one_pair = analyze_ads_performance(28947, click, mda_idx=123)

In [54]:
media_portfolio[media_portfolio['mda_idx'].isin([246])]

,Unnamed: 0,mda_idx,user_count,total_clicks,total_conversions,first_click,last_click,days_active,daily_avg_conversions,LARGE,...,domain_비영리/공공_pct,domain_생활_pct,domain_식당/카페_pct,domain_식음료_pct,domain_운동/스포츠_pct,domain_운세_pct,domain_의료/건강_pct,domain_채용_pct,domain_커머스_pct,expected_total_profit
13,13,246,498,1658,1013,2025-07-26 00:10:46,2025-08-25 04:13:54,31,32.677419,53,...,3.305785,5.785124,0.826446,3.305785,1.652893,4.132231,3.305785,4.132231,9.090909,77757


In [65]:
media_performance[media_performance['mda_idx'].isin([667, 496, 58, 87])]

,Unnamed: 0.1,Unnamed: 0,mda_idx,user_count,total_clicks,total_conversions,first_click,last_click,days_active,daily_avg_conversions,LARGE,MEDIUM,MEGA,SMALL,total_ads,MEGA_ratio,LARGE_ratio,MEDIUM_ratio,SMALL_ratio,ads_category_0_pct,ads_category_1_pct,ads_category_2_pct,ads_category_3_pct,ads_category_4_pct,ads_category_5_pct,ads_category_6_pct,ads_category_7_pct,ads_category_8_pct,ads_category_10_pct,ads_category_11_pct,ads_category_13_pct,ads_os_type_1_pct,ads_os_type_2_pct,ads_os_type_3_pct,ads_os_type_7_pct,domain_게임_pct,domain_교육_pct,domain_금융_pct,domain_기타_pct,domain_미디어/컨텐츠_pct,domain_뷰티_pct,domain_비영리/공공_pct,domain_생활_pct,domain_식당/카페_pct,domain_식음료_pct,domain_운동/스포츠_pct,domain_운세_pct,domain_의료/건강_pct,domain_채용_pct,domain_커머스_pct,expected_total_profit,conversion_rate,last_click_dt,classification,basic_classification
9,9,9,58,169122,482794,155827,2025-07-26 00:01:29,2025-08-25 11:24:04,31,5026.677419354839,12,0,14,0,26,53.8,46.2,0.0,0.0,0.000000000000,3.846153846154,34.615384615385,0.000000000000,0.0,11.538461538462,0.0,3.846153846154,15.384615384615,30.769230769231,0.000000000000,0.0,0.000000000000,53.846153846154,0.000000000000,46.153846153846,46.153846153846,0.000000000000,26.923076923077,3.846153846154,3.846153846154,0.000000000000,0.000000000000,3.846153846154,0.0,0.000000000000,0.000000000000,7.692307692308,3.846153846154,0.000000000000,3.846153846154,3884970,0.322760846241,2025-08-25 11:24:04,안정공급형(후보),안정공급형(후보)
11,11,11,87,30294,64349,21173,2025-07-27 11:01:31,2025-08-25 00:11:49,29,730.103448275862,2,0,4,0,6,66.7,33.3,0.0,0.0,0.000000000000,16.666666666667,16.666666666667,16.666666666667,0.0,0.000000000000,0.0,0.000000000000,50.000000000000,0.000000000000,0.000000000000,0.0,0.000000000000,50.000000000000,0.000000000000,50.000000000000,33.333333333333,0.000000000000,0.000000000000,0.000000000000,16.666666666667,0.000000000000,0.000000000000,0.000000000000,0.0,16.666666666667,0.000000000000,16.666666666667,0.000000000000,0.000000000000,16.666666666667,640407,0.329033862220,2025-08-25 00:11:49,안정공급형(후보),안정공급형(후보)
46,46,46,496,80048,127975,26418,2025-07-26 00:06:05,2025-08-25 11:24:01,31,852.193548387097,24,8,22,0,54,40.7,44.4,14.8,0.0,16.666666666667,11.111111111111,20.370370370370,11.111111111111,0.0,3.703703703704,0.0,9.259259259259,25.925925925926,1.851851851852,0.000000000000,0.0,0.000000000000,66.666666666667,7.407407407407,25.925925925926,25.925925925926,1.851851851852,25.925925925926,1.851851851852,7.407407407407,1.851851851852,5.555555555556,9.259259259259,0.0,5.555555555556,0.000000000000,1.851851851852,3.703703703704,1.851851851852,7.407407407407,1639389,0.206430943544,2025-08-25 11:24:01,안정공급형,안정공급형
70,70,70,667,9048,50811,16049,2025-07-26 00:02:45,2025-08-25 11:18:48,31,517.709677419355,43,28,18,0,89,20.2,48.3,31.5,0.0,0.000000000000,5.617977528090,13.483146067416,4.494382022472,0.0,26.966292134831,0.0,0.000000000000,16.853932584270,29.213483146067,3.370786516854,0.0,21.348314606742,26.966292134831,5.617977528090,46.067415730337,39.325842696629,0.000000000000,19.101123595506,1.123595505618,7.865168539326,2.247191011236,2.247191011236,7.865168539326,0.0,1.123595505618,1.123595505618,8.988764044944,3.370786516854,2.247191011236,3.370786516854,1135899,0.315856802661,2025-08-25 11:18:48,안정공급형,안정공급형
